# Web Based Data Project Python Script

### Importing Libraries and connecting to the server

In [116]:
from pymed import PubMed
import pandas as pd
import scispacy
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from nltk import word_tokenize
import csv
pubmed = PubMed(email="arthur.hughes27@outlook.com")

### Find PubMed articles with keyword "cardiovascular disease" and the available molecules

In [140]:
query = "cardiovascular disease, air pollution[TITLE]"
results= pubmed.query(query, max_results=2000)  #save 50 results 
articleList= []
for article in results:   #for loop to collect articles
  articleDict = article.toDict()
  articleList.append(articleDict)

In [141]:
df= pd.DataFrame(articleList) #save to dataframe
#df.head(50)
df.abstract
#first abstract

0       Telomeres are repetitive DNA-protein sequences...
1       Household air pollution (HAP) due to the use o...
2       The link between air pollution-attributed card...
3       Stroke remains the second cause of death world...
4       Although environmental exposure such as air po...
                              ...                        
1995    This article briefly summarizes the epidemiolo...
1996    The authors analyzed whether there is an assoc...
1997    The authors analyzed daily mortality data in S...
1998    We used daily time-series analysis to evaluate...
1999    The analysis of short term relation between da...
Name: abstract, Length: 2000, dtype: object

### NLP - Find potential links

In [145]:
key_sentences=[]
search_keywords=['associated']
for abstract in df.abstract:
    if abstract is not None:
        sentences = abstract.split(".")
        for sentence in sentences:
            if sum(1 for word in search_keywords if word in sentence)>0:
                key_sentences.append(sentence.lower())

In [146]:
potential_links=[]
search_keywords=['NO2','O3','particulate','pollutant','air pollution','SO2','CO2','PM2.5','PM10','CO','Carbon','Oxygen']
for element in key_sentences:
    if sum(1 for word in search_keywords if word in element)>0:
        potential_links.append(element)
df2=pd.DataFrame(potential_links)
df2.to_csv('sentences.csv',index=False)

In [147]:
df2

,0
0,"in particular, air pollution was associated w..."
1,\nan increased exposure to any class of air po...
2,long-term exposure to air pollution has been a...
3,\nto investigate changes in serum metabolites ...
4,27 μg/m\nour study suggested that long-term ex...
...,...
784,"in the elderly, a 3-4% increase in daily deat..."
785,"in addition, it seems that older age groups a..."
786,air pollution was associated with each of the...
787,"in general, the gases, particularly co, but n..."
